## CV17 - Human Pose Estimation with Keypoint detection
https://darkpgmr.tistory.com/77
https://darkpgmr.tistory.com/83?category=460965

키포인트를 찾는 방법
1. Top-down
   object detection 사용하여 crop한 이미지 내에서 찾는다.  
   사람이 많이 등장할 경우 느리다.  
2. Bottom-up
   모든 키포인트를 먼저 찾아서 사람의 형태로 맞추어 나간다.  
   detect하지 않기 때문에 속도 저하가 없지만 모든 키포인트 찾아내기 때문에 성능이 떨어진다.  
   
등장 하는 사람의 수에 따라 방법을 정하면 된다.

### human ketpoint detection
https://github.com/Team-Neighborhood/Kalman-Filter-Image  
키포인트 찾는데 어려운 점: invisible, occlusions, clothing, lighting change 등  
https://nanonets.com/blog/human-pose-estimation-2d-guide/  
인체는 단위 부분으로 나눌 수 있고 그 부분은 서로 연결성을 가지고 있다. 3D에서는 구분이 되지만 2D에서는 구분이 잘 안된다.  
각 부분의 complex joint relationship의 mixture model로 키포인트를 표현한다.  
https://www.cs.cmu.edu/~deva/papers/pose_pami.pdf

DeepPose  
Toshev and Szegedy가 처음으로 딥러닝 기반 keypoint localization 모델 제안  
https://arxiv.org/pdf/1312.4659.pdf  
초기의 pose estimation은 x, y좌표를 직접 예측하는 position regression문제로 인식하였다. (L2 loss)  

Efficient object localization Using convolutional network  
x, y좌표를 직접 예측하기보다는 keypoint가 존재할 확률 분포가 높은 곳을 찾자.  
x, y좌표를 중심으로 한 heatmap를 학습하도록 한다. 엄청난 성능 향상을 가져온다.  
https://arxiv.org/pdf/1411.4280.pdf  

Convolutional Pose Machines  
https://arxiv.org/pdf/1602.00134.pdf  
이전에는 crop 연산 등으로 비연속적인 stage로 나뉘어져 있어서 학습 과정을 반복하는 단점이 있었다.  
end-to-end, 한 번에 학습할 수 있다.
stage1에서 image feature를 생성, stage2부터는 keypoint를 예측한다.  
각 stage의 g1, g2 함수는 모두 heatmap를 출력하여 재사용하며 weight sharing을 하게 된다.  
stage2을 반복하면 receptive field가 넓어지면서 joint를 재조정하면서 성능이 향상되었다.  

Stacked Hourglass Network  
feature map upsampling, residual connection이 주요점이다.  
pooling으로 이미지의 global feature를 찾고 upsampling으로 local feature를 고려한다.  
기본 구조가 hourglass형태이며 이를 여러 개 쌓는다.  
MPII에서 처음으로 PCKh@0.5기준 90%이상 나옴.

SimpleBaseline
아주 간단한 encoder-decoder 구조, 73.7%의 mAP 성능이 나옴, resnet50만 사용한 간단한 구조이지만 hourglass를 넘겼다.  
Deep High-Resolution Network(HRNet)  
SOTA에 가까운 모델, SimpliBaseline과 같은 저자라 같은 철학을 보여줌.  
multi-stage가 아닌 1stage구조를 고수하는 간단한 모델을 지향
hourglass와 simplebaseline의 공통점과 차이점은?  
encoder(high) -> low -> decoder(high) 과정에서 최초의 정보를 최종까지 얼마나 많이 효율적으로 가지고 갈 수 있을까?  
down sample layer: 작아진 layer feature를 upsampling하여 원본 해상도 크기에 적용하는 방법  
MSE loass 이용  
https://github.com/leoxiaobin/deep-high-resolution-net.pytorch 원저작자 github

### SimpleBaseline 구조 코드로 이해하기
https://arxiv.org/pdf/1804.06208.pdf  
encoder: resnet을 backbone  
decoder: deconv - bn - relu, 256filter size, 4x4 kernel, stride 2  
https://github.com/Microsoft/human-pose-estimation.pytorch  원저작자의 github repo
모델 부분 : https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/lib/models/pose_resnet.py  
torch.nn == keras.layers  
29번째 줄 basicblock == keras.modes: https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/lib/models/pose_resnet.py#L29  
forward함수: https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/lib/models/pose_resnet.py#L42  
4개의 residual block: https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/lib/models/pose_resnet.py#L157  
forward함수: https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/lib/models/pose_resnet.py#L234  
resnet - deconv_layers - final_layer  
deconv layer: https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/lib/models/pose_resnet.py#L219  
파라미터 config 파일: https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/experiments/coco/resnet50/256x192_d256x3_adam_lr1e-3.yaml#L23  


In [1]:
# simplebaseline tf 버전
import os

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet')

In [2]:
# deconv module
upconv1 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn1 = tf.keras.layers.BatchNormalization()
relu1 = tf.keras.layers.ReLU()
upconv2 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn2 = tf.keras.layers.BatchNormalization()
relu2 = tf.keras.layers.ReLU()
upconv3 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn3 = tf.keras.layers.BatchNormalization()
relu3 = tf.keras.layers.ReLU()

In [3]:
# deconv module 함수로 만들기
def _make_deconv_layer(num_deconv_layers):
    seq_model = tf.keras.models.Sequential()
    for i in range(num_deconv_layers):
        seq_model.add(tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same'))
        seq_model.add(tf.keras.layers.BatchNormalization())
        seq_model.add(tf.keras.layers.ReLU())
    return seq_model

upconv = _make_deconv_layer(3)

In [4]:
final_layer = tf.keras.layers.Conv2D(17, kernel_size=(1,1), padding='same')

In [5]:
def _make_deconv_layer(num_deconv_layers):
    seq_model = keras.models.Sequential()
    for i in range(num_deconv_layers):
        seq_model.add(tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same'))
        seq_model.add(tf.keras.layers.BatchNormalization())
        seq_model.add(tf.keras.layers.ReLU())
    return seq_model

resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet')
upconv = _make_deconv_layer(3)
final_layer = tf.keras.layers.Conv2D(1, kernel_size=(1,1), padding='same')

# input :  192x256
# output : 48x64
inputs = keras.Input(shape=(256, 192, 3))
x = resnet(inputs)
x = upconv(x)
out = final_layer(x)
model = keras.Model(inputs, out)

model.summary()

# np_input = np.zeros((1,256,192,3), dtype=np.float32)
np_input = np.random.randn(1,256,192,3)
np_input = np.zeros((1,256,192,3), dtype=np.float32)
tf_input = tf.convert_to_tensor(np_input, dtype=np.float32)
print (tf_input.shape) # TensorShape([1,256,192,3])

tf_output = model(tf_input)

print (tf_output.shape)
print (tf_output[0,:10,:10,:10])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 256, 192, 3)]     0         
_________________________________________________________________
resnet50 (Model)             multiple                  23587712  
_________________________________________________________________
sequential_1 (Sequential)    (None, 64, 48, 256)       10489600  
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 48, 1)         257       
Total params: 34,077,569
Trainable params: 34,022,913
Non-trainable params: 54,656
_________________________________________________________________
(1, 256, 192, 3)
(1, 64, 48, 1)
tf.Tensor(
[[[-2.37211911e-03]
  [ 6.58433978e-03]
  [-1.95678752e-02]
  [-6.39829272e-03]
  [-6.63550338e-03]
  [-9.39821638e-03]
  [ 3.42872227e-03]
  [-7.97738321e-04]
  [ 2.01119343e-03]
  [ 3.12840356e-03]